# How Well Can LLMs Negotiate? NegotiationArena. Platform and Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Z1M97k4GEf2_v48cdA96BANTAp0yK2IM?usp=sharing)
[![Arxiv Preprint](https://img.shields.io/badge/arXiv-2402.05863-0.svg)](https://arxiv.org/abs/2402.05863)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)
[![GitHub](https://img.shields.io/badge/Code-GitHub-blue?logo=github)](https://github.com/vinid/NegotiationArena)


NegotiationArena is a platform to develop games to better understand how agents interact with each other to solve engotiation problems. In the image below you see an example of negotiation between the agents.

<div align="center">
<img src="https://github.com/vinid/NegotiationArena/blob/main/figures/negotiation_intro.jpg?raw=true" width=400>
</div>



## Who is this tutorial for

**If you want to run one of the games. The game we are going to run is the BuySell game.**


From the paper:


> We introduce a seller and buyer game involving two agents, one looking to sell a set of resources and one looking to buy them, similar to other approaches in the literature. We imbue agents with some beliefs about the object being sold, but unlike the ultimatum game, the seller and buyer game is an incomplete information game, i.e., players do not have complete information about other players (e.g., their beliefs). Only the seller is aware of the production cost of the object, and only the buyer is assigned and is aware of their willingness to pay for the object. Given these beliefs, the seller and the buyer are prompted to sell and buy the object, respectively. The seller starts first: reproducing a scenario in which the object is already on sale.

Here we run the BuySell game.




### **Let's Start!**

We now install and load what we need to run the games.




In [ ]:
%%capture

!git clone https://github.com/vinid/NegotiationArena/
!pip install -r NegotiationArena/requirements.txt

In [ ]:
import sys
from dotenv import load_dotenv

sys.path.append('NegotiationArena/')

In [ ]:
from negotiationarena.agents.chatgpt import ChatGPTAgent
from negotiationarena.game_objects.resource import Resources
from negotiationarena.game_objects.goal import BuyerGoal, SellerGoal
from negotiationarena.game_objects.valuation import Valuation
from negotiationarena.constants import AGENT_ONE, AGENT_TWO, MONEY_TOKEN
import traceback
from games.buy_sell_game.game import BuySellGame


## **Important Background**


We first need to introduce a game prompt. This prompt will describe the rules of the game.

By default, we support a TAG/XML-like language for communcation. Thus we will ask all the agents to follow this formatting.

```
<answer> REJECT </answer>
<message> I don't like your trade, here's my counter offer. </message>
<trade> Player RED Gives X:50 | Player Blue Gives Y:40 </trade>
```



## **Prompt**

Under this cell you have the entire prompt of the game. The prompt you see is the prompt of the seller. The buyer's one is slightly different (the buyer has a different goal)




```You are playing game where you are buying or selling an object. There is only one object for sale/purcahse.

Player RED is going to sell one object. Player BLUE gives ZUP to buy resources.

RULES:

1. You must always respond with:

    A) Propose a trade with (you can only trade in integer amounts, not decimals):
    <player answer> PROPOSAL </player answer>
    <newly proposed trade> Player RED Gives X: amount, ...| Player BLUE Gives ZUP: amount </newly proposed trade>

    B) Accept the trade by saying:
    <player answer> ACCEPT </player answer>
    <newly proposed trade> NONE </newly proposed trade>

    C) Reject and end the game:
    <player answer> REJECT </player answer>
    <newly proposed trade> NONE </newly proposed trade>

    Note: The game will end if one of the players ACCEPT OR REJECT. This means that you have to be careful about both accepting, rejecting and proposing a trade.

2. You are allowed at most 4 proposals of your own to complete the game, after which you can only reply with ACCEPT or REJECT.
DO NOT propose a new trade after 4 proposals. Your limit for proposals is 4.

3. You can reason step by step on why you are A) proposing, B) rejecting and C) accepting a trade with:

<reason> [add reasoning] </reason> add as much text as you want

This information will not be sent to the other player. It is just for you to keep track of your reasoning.

4. At each turn send messages to each other by using the following format:

<message>your message here</message>

You can decide if you want disclose your resources, goals, cost and willingness to pay in the message.


Here is what you have access to:

Object that is being bought/sold: X
<my resources> X: 1 </my resources>
<my goals> Sell resources for <ZUP>. It costed X: 40 ZUP to produce the resources </my goals>,


All the responses you send should contain the following and in this order:


<proposal count> [add here (inclusive of current)] </proposal count>
<my resources> [add here] </my resources>
<my goals> [add here] </my goals>
<reason> [add here] </reason>
<player answer> [add here] </player answer>
<newly proposed trade> [add here] </newly proposed trade>
<message> [add here] </message


Please be sure to include all.
```

## **Setup**

**You need to upload an environment file called "env" that should have the following content:**

```
OPENAI_API_KEY=
```

Your colab session is local to your account, but if you don't like putting your OpenAI key in colab i'd just create a throwaway one just for the sake of playing with the example.


**Be mindful** of what you do! These games should end very quickly and does should not cost much, but you might end up paying more if you make the agents chat for 1000 turns!

In [ ]:
# the project assumes you have your environment variable in an env file.
load_dotenv("env")

True

Note, **some games might break because gpt-4 might forget to follow instructions.** That is why we currently have a **try/catch** statement.

## **Base Game**

Now let's look at the base game! We have added some comments on all the lines to make explanations of what's happening clearer.

In [ ]:
try:

    # we implement two chatgpt4 agents.
    a1 = ChatGPTAgent(agent_name=AGENT_ONE, model="gpt-4-1106-preview")
    a2 = ChatGPTAgent(agent_name=AGENT_TWO, model="gpt-4-1106-preview")

    # buy sell game
    c = BuySellGame(
        players=[a1, a2], # our two players
        iterations=10, # they can interact for 10 turns
        player_goals=[

            # they have different goals, with a different valuation for those goals
            SellerGoal(cost_of_production=Valuation({"X": 40})),
            BuyerGoal(willingness_to_pay=Valuation({"X": 60})),
        ],

        # player 1 has X, the object to sell.
        # player 2 has money, that here is identified by the currency ZUP.
        player_starting_resources=[
            Resources({"X": 1}),
            Resources({MONEY_TOKEN: 100}),
        ],

        # we assign roles here, this have a reference in the prompt.
        player_conversation_roles=[
            f"You are {AGENT_ONE}.",
            f"You are {AGENT_TWO}.",
        ],

        # here we can set social behaviors
        player_social_behaviour=[
            "",
            "",
        ],
        log_dir="../example_logs/buysell",
    )

    c.run()
except Exception as e:
    exception_type = type(e).__name__
    exception_message = str(e)
    stack_trace = traceback.format_exc()

    # Print or use the information as needed
    print(f"Exception Type: {exception_type}")
    print(f"Exception Message: {exception_message}")
    print(f"Stack Trace:\n{stack_trace}")


State:
current_iteration : 1
turn : 0
player_complete_answer : <proposal count>1</proposal count>
<my resources> X: 1 </my resources>
<my goals> Sell resources for ZUP. It costed X: 40 ZUP to produce the resources </my goals>
<reason>I want to start with a proposal that allows me to make a profit considering the production cost of 40 ZUP for my resource. I'll propose a higher amount to leave room for negotiation.</reason>
<player answer>PROPOSAL</player answer>
<newly proposed trade> Player RED Gives X: 1 | Player BLUE Gives ZUP: 50 </newly proposed trade>
<message>Hello Player BLUE, I have a high-quality resource X available for trade. Considering the production costs and the value it offers, I propose an exchange where you give me 50 ZUP for my resource X. Looking forward to your counteroffer or acceptance!</message>

State:
current_iteration : 2
turn : 1
player_complete_answer : <proposal count> 1 </proposal count>
<my resources> ZUP: 100 </my resources>
<my goals> Buy resources wit

If your run the code above, you should have seen some interaction between the different agents. You can see them interacting and making offers.

## **Running A Different Scenario**

You can re run the game, by changing the valuation of the object.  Let's try to make the valuations of the object equal for the two agents.

In [ ]:
try:
    a1 = ChatGPTAgent(agent_name=AGENT_ONE, model="gpt-4-1106-preview")
    a2 = ChatGPTAgent(agent_name=AGENT_TWO, model="gpt-4-1106-preview")

    c = BuySellGame(
        players=[a1, a2],
        iterations=10,
        player_goals=[
            SellerGoal(cost_of_production=Valuation({"X": 50})),
            BuyerGoal(willingness_to_pay=Valuation({"X": 50})),
        ],
        player_starting_resources=[
            Resources({"X": 1}),
            Resources({MONEY_TOKEN: 100}),
        ],
        player_conversation_roles=[
            f"You are {AGENT_ONE}.",
            f"You are {AGENT_TWO}.",
        ],
        player_social_behaviour=[
            "",
            "",
        ],
        log_dir="../example_logs/buysell",
    )

    c.run()
except Exception as e:
    exception_type = type(e).__name__
    exception_message = str(e)
    stack_trace = traceback.format_exc()

    # Print or use the information as needed
    print(f"Exception Type: {exception_type}")
    print(f"Exception Message: {exception_message}")
    print(f"Stack Trace:\n{stack_trace}")


### **Social Behavior**

You can also explore different social behaviors, these are added to the prompt and should change the way the agents interact.

In [ ]:
try:
    a1 = ChatGPTAgent(agent_name=AGENT_ONE, model="gpt-4-1106-preview")
    a2 = ChatGPTAgent(agent_name=AGENT_TWO, model="gpt-4-1106-preview")

    c = BuySellGame(
        players=[a1, a2],
        iterations=10,
        player_goals=[
            SellerGoal(cost_of_production=Valuation({"X": 40})),
            BuyerGoal(willingness_to_pay=Valuation({"X": 60})),
        ],
        player_starting_resources=[
            Resources({"X": 1}),
            Resources({MONEY_TOKEN: 100}),
        ],
        player_conversation_roles=[
            f"You are {AGENT_ONE}.",
            f"You are {AGENT_TWO}.",
        ],

        # new social behaviors
        player_social_behaviour=[
            "",
            "You don't like the other player. You insult them. You want to pay less because you know the object is low quality.",
        ],
        log_dir="../example_logs/buysell",
    )

    c.run()
except Exception as e:
    exception_type = type(e).__name__
    exception_message = str(e)
    stack_trace = traceback.format_exc()

    # Print or use the information as needed
    print(f"Exception Type: {exception_type}")
    print(f"Exception Message: {exception_message}")
    print(f"Stack Trace:\n{stack_trace}")
